#### Note - This model is actually used for this project and uses the Darknet-19 forward prop outlined in the YOLO_v2 paper.

## YOLO_v2 inspired model

- This model takes in input images of shape (448,448,3) and produces output of shape (14,14,14). In looking at the output of my k-means clustering step in my minibatch data cleaning notebook, I decided to have 2 (1,1,7) encodings such that the model can learn the difference between the dimensions of open and closed palms.

- I will be using stochastic gradient descent with adam optimization, but with a manually interatively decreasing learning rate, unlike the set diminishing learning rate promoted by the YOLO_v2 model.

- This model is essentially the Darknet-19 model, which I chose over the Darknet-53 resnet model strictly to decrease the time needed for model training.

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from keras import backend as K
import matplotlib.pyplot as plt
import latex
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
# Loading data
X = np.load("../../data/dinorunner/images_448.npy")
y = np.load("../../data/dinorunner/encodings_448.npy")
print(X.shape)
print(y.shape)

(396, 448, 448, 3)
(396, 14, 14, 7)


In [3]:
# Creating testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=1)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(376, 448, 448, 3)
(376, 14, 14, 7)
(20, 448, 448, 3)
(20, 14, 14, 7)


### Tensorflow Placeholders

In [4]:
# Placeholder values for input X,y data
def get_placeholders(x_h,x_w,x_c,y_h,y_w,y_c):
    """
    x_h: Height for x input 
    x_w: Width for x input
    x_c: Channels for x input
    y_h: Height for y input
    y_w: Width for y input
    y_c: Channels for y input
    """
    X = tf.placeholder(tf.float32, name="X", shape=(None,x_h,x_w,x_c))
    y = tf.placeholder(tf.float32, name="y", shape=(None,y_h,y_w,y_c))
    return X,y

In [5]:
# Testing placeholders
tf.reset_default_graph()
with tf.Session() as sess:
    X,y = get_placeholders(448,448,3,14,14,7)
    print("X shape:",X.shape)
    print("y shape:",y.shape)

X shape: (?, 448, 448, 3)
y shape: (?, 14, 14, 7)


### Tensorflow Forward Propagation

In [6]:
# Defining constant layer for 2d convolution, batch norm, and activation
def conv(the_input,layer,f,ks):
    """
    the_input: the layer which will be used as input in conv layer
    layer: specifies the layer number for naming sections of graph
    f (filters): the number of filters to be used for conv layer
    ks (kernel_size): kernel size for conv2d layer
    Note - conv2d layers all use padding
    """
    layer = str(layer)
    Z = tf.layers.conv2d(the_input,filters=f,kernel_size=[ks,ks],strides=(1,1),padding="same",name="Z"+layer,kernel_initializer=tf.contrib.layers.xavier_initializer(seed=0))
    Bn = tf.layers.batch_normalization(Z,name="Bn"+layer)
    A = tf.nn.leaky_relu(Bn,alpha=0.1,name="A"+layer)
    return A

In [7]:
# Building the forward pass based on Darket-19
# Note - forward pass will use leaky_relu
def forward_pass(X):
    input_layer = tf.reshape(X,[-1,448,448,3]) # Input shape of images
    S1 = conv(input_layer,1,32,3)
    P1 = tf.layers.max_pooling2d(S1,pool_size=[2,2],strides=2,padding="valid",name="P1") # 224x224
    S2 = conv(P1,2,64,3)
    P2 = tf.layers.max_pooling2d(S2,pool_size=[2,2],strides=2,padding="valid",name="P2") # 112x112
    S3 = conv(P2,3,128,3)
    S4 = conv(S3,4,64,1)
    S5 = conv(S4,5,128,3)
    P5 = tf.layers.max_pooling2d(S5,pool_size=[2,2],strides=2,padding="valid",name="P5") # 56x56
    S6 = conv(P5,6,256,3)
    S7 = conv(S6,7,128,1)
    S8 = conv(S7,8,256,3)
    P8 = tf.layers.max_pooling2d(S8,pool_size=[2,2],strides=2,padding="valid",name="P8") # 28x28
    S9 = conv(P8,9,512,3)
    S10 = conv(S9,10,256,1)
    S11 = conv(S10,11,512,3)
    S12 = conv(S11,12,256,1)
    S13 = conv(S12,13,512,3)
    P13 = tf.layers.max_pooling2d(S13,pool_size=[2,2],strides=2,padding="valid",name="P13") #14x14
    S14 = conv(P13,14,1024,3)
    S15 = conv(S14,15,512,1)
    S16 = conv(S15,16,1024,3)
    S17 = conv(S16,17,512,1)
    S18 = conv(S17,18,2014,3)
    # Final layer - no batch norm, linear activation
    S19 = tf.layers.conv2d(S18,filters=14,kernel_size=[1,1],strides=(1,1),padding="valid",name="S19",activation=None)
    return S19

In [8]:
# Testing forward prop
tf.reset_default_graph()
with tf.Session() as sess:
    np.random.seed(1)
    X,y = get_placeholders(448,448,3,14,14,7)
    Z19 = forward_pass(X) # Computation graph
    init = tf.global_variables_initializer()
    sess.run(init)
    aZ = sess.run(Z19,feed_dict={X:np.random.randn(1,448,448,3),y:np.random.randn(1,14,14,7)})
    print("Z shape:", str(aZ.shape))

Z shape: (1, 14, 14, 14)


### Tensorflow Cost Function

Using the predicted box [shape(1,1,7)] with the highest IoU score to determine the prediction cost for a given image.

Note - I deviate from the YOLO function function in that for all of the cells that don't hold the actual prediction for the bounding box, I penalize ALL of the confidence predictions (2 per cell in this case). In order to compensate for this change and not greatly increase the cost from just wrong confidence scores I decrease the lambda<sub>noobj</sub> from 0.5 to 0.25. This is due to the fact that I am factoring in 2 times the previous number of confidence scores.

In [9]:
# Returns the sum of all of the confidence scores corresponding w/ cells that don't have prediction
def get_false_conf(Z,mask):
    conf_1 = tf.square(tf.boolean_mask(Z[:,:,:,0:1],mask))
    conf_2 = tf.square(tf.boolean_mask(Z[:,:,:,7:8],mask))
    conf_sum = tf.add(tf.reduce_sum(conf_1),tf.reduce_sum(conf_2))
    return conf_sum

In [10]:
# Returns the values with a specific mask applied to it
def get_box_values(box,mask):
    """
    Index:
    0: confidence there is an object in cell, 1: mid_x, 2: mid_y, 
    3: width, 4: length, 5: prob_open_palm, 6: prob_close_palm
    """
    confidence = tf.boolean_mask(box[:,:,:,0:1],mask)
    mid_x = tf.boolean_mask(box[:,:,:,1:2],mask)
    mid_y = tf.boolean_mask(box[:,:,:,2:3],mask)
    width = tf.boolean_mask(box[:,:,:,3:4],mask)
    height = tf.boolean_mask(box[:,:,:,4:5],mask)
    prob_open = tf.boolean_mask(box[:,:,:,5:6],mask)
    prob_closed = tf.boolean_mask(box[:,:,:,6:7],mask)
    box = {"co":confidence, "mx":mid_x,"my":mid_y,"w":width,"h":height,"d":prob_open,"c":prob_closed}
    return box

In [11]:
# Returns the IoU
def get_iou(box1, box2):
    """
    box1 - coordinates (x1, y1, x2, y2)
    box2 - coordinates (x1, y1, x2, y2)
    """
    xi1 = tf.maximum(box1["x1"], box2["x1"])
    yi1 = tf.maximum(box1["y1"], box2["y1"])
    xi2 = tf.minimum(box1["x2"], box2["x2"])
    yi2 = tf.minimum(box1["y2"], box2["y2"])
    inter_area = tf.maximum((xi2 - xi1),0) * tf.maximum((yi2 - yi1),0) # no neg bounding box

    box1_area = (box1["x2"]-box1["x1"]) * (box1["y2"] - box1["y1"])
    box2_area = (box2["x2"]-box2["x1"]) * (box2["y2"] - box2["y1"])
    union_area = box1_area + box2_area - inter_area
    iou = inter_area / union_area
    
    return tf.reshape(iou,[])

In [12]:
# Returns the (x1,y1),(x2,y2) coordinates for each bounding box as a dict
def get_xy(box_v):
    mid_x = box_v["mx"]
    mid_y = box_v["my"]
    width = box_v["w"]
    height = box_v["h"]
    width = width * 448
    height = height * 448
    mid_x = mid_x * 32 + 224
    mid_y = mid_y * 32 + 224
    
    x1 = mid_x - (1/2*width)
    x2 = mid_x + (1/2*width)
    y1 = mid_y - (1/2*height)
    y2 = mid_y + (1/2*height)
    
    box_xy = {"x1":x1,"x2":x2,"y1":y1,"y2":y2}
    return box_xy

In [13]:
# This returns the information associated with the bounding box prediction with the max IoU
def get_max_iou(box1, box2, y, mask):
    """
    b1_v: bounding box values associated with box1
    b2_v: bounding box values associated with box2
    y_v: bounding box values associated with y
    """
    b1_v = get_box_values(box1,mask)
    b2_v = get_box_values(box2,mask)
    y_v = get_box_values(y,mask)
    # These new coordinates will be used to get the IoU
    b1_xy = get_xy(b1_v)
    b2_xy = get_xy(b2_v)
    y_xy = get_xy(y_v)
    # Getting the Iou for each bounding box prediction
    b1_iou = get_iou(b1_xy, y_xy)
    b2_iou = get_iou(b2_xy, y_xy)
    # Comparing the ious to determine which guess is the ground truth prediction
    def b1(): return b1_v, b2_v
    def b2(): return b2_v, b1_v
    highest_iou_values, lowest_values = tf.cond(tf.less(b1_iou,b2_iou), b2, b1)
    return highest_iou_values, lowest_values

In [14]:
# Stochastic gradient descent so input of shape (1,14,14,14)
# Heavily penalizes negative predictions for height and weight
# Bounding box prediction based on guess with the highest IoU
def cost_function(Z,y,coord=5,noobj=0.25):
    """
    Z - shape (?,14,14,14)
    y - shape (?,14,14,7)
    """
    c_mask_true = y[:,:,:,0:1] > 0 # which cell has the encoding
    c_mask_false = y[:,:,:,0:1] < 1
    
    box1 = Z[:,:,:,0:7] # First guess
    box2 = Z[:,:,:,7:14] # Second guess
    y_v = get_box_values(y,c_mask_true) # values for y for cell with object
    m_v,lowest_values = get_max_iou(box1,box2,y,c_mask_true) # values for highest IoU guess
    false_conf_score = get_false_conf(Z,c_mask_false) # sum of false confidence predictions
    #false_conf_score = false_conf_score + tf.reshape(tf.square(lowest_values["co"]),[])
    
    # penalizing width,length predictions if negative
    m_v["w"] = tf.sqrt(tf.maximum(m_v["w"],0.0))
    m_v["l"] = tf.sqrt(tf.maximum(m_v["h"],0.0))
    
    y_v["w"] = tf.sqrt(y_v["w"])
    y_v["l"] = tf.sqrt(y_v["h"])
    
    # correspond to individual summations of the cost function:
    part1 = coord * tf.reduce_sum(tf.square(y_v["mx"]-m_v["mx"])+tf.square(y_v["my"]-m_v["my"]))
    part2 = coord * tf.reduce_sum(tf.square(y_v["w"]-m_v["w"])+tf.square(y_v["l"]-m_v["l"]))
    part3 = tf.reduce_sum(tf.square(y_v["co"]-m_v["co"]))
    part4 = noobj * false_conf_score
    part5 = tf.reduce_sum(tf.add(tf.square(y_v["d"]-m_v["d"]),tf.square(y_v["c"]-m_v["c"])))
    total_cost = part1 + part2 + part3 + part4 + part5

    return total_cost

In [15]:
# Testing cost function
# predicted cost w/ rounding error is 1.4396
ay = np.zeros((1,14,14,7))
ay[0,0,0,:] = np.array([1,0.5,0.5,0.25,0.25,1,0]) # top left corner
az = np.zeros((1,14,14,14))
az[0,0,0,0:7] = np.array([0.8,0.35,0.35,0.2,0.2,0.8,0.2]) # PRED 1
az[0,0,0,7:] = np.array([0.5,0.4,0.4,0.22,0.22,0.8,0.2]) # PRED 2
az[0,0,1,0] = 1
az[0,1,0,0] = 1
az[0,1,0,7] = 1
az[0,2,0,7] = 1

with tf.Session() as sess:
    y = tf.placeholder(tf.float32,shape=(None,14,14,7))
    Z = tf.placeholder(tf.float32,shape=(None,14,14,14))
    aCost = cost_function(Z,y)
    init = tf.global_variables_initializer()
    sess.run(init)
    tot = sess.run(aCost,feed_dict={Z:az,y:ay})
    print(tot)

1.4395843


### Tensorflow Model

In [21]:
# Building and training YOLO model using stochastic gradient descent
def model(X_train,y_train,lr=0.001,num_epochs=50,print_cost=True):
    tf.reset_default_graph() # resetting graph
    tf.set_random_seed(1)
    seed=0
    costs=[]
    x_h = X_train[0].shape[0]
    x_w = X_train[0].shape[1]
    x_c = X_train[0].shape[2]
    y_h = y_train[0].shape[0]
    y_w = y_train[0].shape[1]
    y_c = y_train[0].shape[2]
    m = X_train.shape[0]
    
    X,y = get_placeholders(x_h,x_w,x_c,y_h,y_w,y_c)
    Z = forward_pass(X)
    cost = cost_function(Z,y)
    optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)
    
    init = tf.global_variables_initializer()
    saver = tf.train.Saver() # to save/load model
    with tf.Session() as sess:
        # Loading saved model
        saver = tf.train.import_meta_graph("../../data/dinorunner/models_stochastic/yolo_model.ckpt.meta")
        saver.restore(sess, "../../data/dinorunner/models_stochastic/yolo_model.ckpt")
        #sess.run(init) # DON'T RUN INIT IF LOADING MODEL
        
        for epoch in range(num_epochs):
            running_cost = 0
            # shuffling training set order each iteration
            seed += 1
            X_train = shuffle(X_train, random_state=seed) 
            y_train = shuffle(y_train, random_state=seed)
            
            for i in range(X_train.shape[0]):
                aX = X_train[i]
                aX.shape = (1,448,448,3)
                aY = y_train[i]
                aY.shape = (1,14,14,7)
                _,temp_cost = sess.run([optimizer,cost], feed_dict={X:aX,y:aY})
                running_cost += temp_cost
                #print(running_cost)
                
            costs.append(running_cost)
            if print_cost and epoch % 1 == 0:
                print("Cost at epoch {}: {}".format(epoch+1,running_cost))
                
        loc = saver.save(sess, "../../data/dinorunner/models_stochastic/yolo_model.ckpt")
        return costs

In [17]:
print(X_train.shape)
print(y_train.shape)

(376, 448, 448, 3)
(376, 14, 14, 7)


In [19]:
acosts = model(X_train,y_train,lr=0.0001,num_epochs=20,print_cost=True)

7.570575714111328
13.302615642547607
20.062905311584473
28.75991153717041
36.11800575256348
38.9745876789093
43.07772374153137
46.80519914627075
50.209975719451904
53.27364492416382
56.57485294342041
59.41601777076721
62.790687084198
66.87264776229858
69.32857036590576
73.78550720214844
75.47061836719513
78.21627962589264
82.81124317646027
85.09737765789032
87.7090402841568
90.2771612405777
95.16017305850983
99.54486382007599
102.73099029064178
105.58709514141083
108.01889884471893
110.50891363620758
113.33190023899078
115.9145804643631
118.95929110050201
121.5904620885849
123.64429032802582
125.89225351810455
128.82098019123077
133.6373678445816
135.5106908082962
137.39238107204437
140.0306099653244
143.5664654970169
145.30394279956818
147.6106961965561
149.21590757369995
151.06431221961975
152.71809697151184
154.90798044204712
158.08949065208435
161.53003573417664
164.20301008224487
167.2007508277893
169.04961395263672
171.74506521224976
173.63829708099365
175.5706752538681
178.78785

176.50345134735107
179.05223512649536
180.90638518333435
183.06228303909302
186.445415019989
190.91281509399414
193.1254756450653
195.62647652626038
198.28501105308533
200.37015628814697
202.759827375412
205.12870573997498
207.37103509902954
209.7599925994873
212.6755199432373
216.03994846343994
219.16877269744873
220.79449677467346
222.98141384124756
224.64178919792175
226.5843780040741
228.24654507637024
231.22694849967957
233.17700147628784
236.0824213027954
238.83450603485107
241.1089699268341
243.02054810523987
245.5049421787262
248.76686930656433
250.88724613189697
254.46788144111633
256.3478217124939
259.6701331138611
261.3453493118286
264.1644170284271
266.83795857429504
269.05954813957214
271.1598174571991
273.5243082046509
275.8712124824524
277.30610966682434
280.06865882873535
282.76938343048096
284.489364862442
287.99713468551636
291.0835118293762
293.1965436935425
295.58388566970825
298.2906856536865
301.3976435661316
305.07799768447876
308.07361364364624
309.6743628978729

363.2016267776489
365.2893922328949
367.20185685157776
369.4662125110626
372.1265215873718
374.9765725135803
377.8471140861511
380.0876739025116
381.91929066181183
383.75827896595
386.22228944301605
388.39469015598297
390.73564088344574
393.3921300172806
395.1523530483246
396.7801058292389
398.2479306459427
400.3782788515091
403.13673174381256
405.1402996778488
406.69639027118683
409.39667904376984
412.58333671092987
416.5152200460434
419.0961478948593
421.06195080280304
423.8672305345535
425.94080793857574
428.05890810489655
429.7811952829361
432.1163240671158
434.842715382576
438.32231986522675
439.84444308280945
443.3192105293274
444.892520070076
448.31348621845245
450.98956167697906
453.3628045320511
455.0676453113556
457.1200611591339
459.28740668296814
461.6828532218933
463.2550085783005
465.468443274498
468.11139929294586
471.23691523075104
474.85270273685455
476.54224622249603
478.6251987218857
480.8483589887619
484.68600714206696
486.6587529182434
488.67147159576416
491.209447

534.336635351181
536.3183178901672
538.626704454422
540.6182264089584
543.742485165596
545.8884793519974
547.8500345945358
549.6804473400116
551.4012632369995
553.405978679657
555.2103452682495
557.8785653114319
560.9432375431061
563.7118570804596
566.4546573162079
568.1517783403397
569.8646359443665
572.6381220817566
576.2555375099182
579.738874912262
582.0912353992462
584.0028977394104
585.4754345417023
588.2355697154999
590.1661077737808
592.5932348966599
595.5924197435379
597.2988133430481
599.7481002807617
602.0021677017212
603.8527548313141
605.7408244609833
607.9662718772888
611.5532562732697
613.8993949890137
615.5661268234253
617.0426349639893
618.4970968961716
620.5878232717514
622.3571063280106
624.432976603508
626.7780455350876
628.8821343183517
630.9871393442154
632.9559409618378
635.1035866737366
637.0729744434357
639.317330121994
642.1410973072052
645.3680572509766
647.0109746456146
649.8379819393158
651.7908182144165
654.9039108753204
656.960569858551
660.2833249568939


710.2621296644211
712.3829239606857
714.2571297883987
717.1858111619949
720.1634584665298
723.4098562002182
725.6736377477646
727.8164275884628
731.0145996809006
732.6871727705002
734.9392145872116
737.2510133981705
739.5813266038895
742.1525000333786
744.8153060674667
747.3091558218002
748.8779658079147
751.6684657335281
753.7592562437057
756.4909576177597
759.2143565416336
762.220898270607
764.3791357278824
767.5129851102829
769.804124712944
771.8567794561386
774.3670791387558
775.9393215179443
778.5580015182495
780.428807258606
783.158224105835
785.0312030315399
787.3542454242706
790.4011261463165
792.5558025836945
795.6706097126007
797.3011918067932
799.9581115245819
802.5654566287994
804.7485492229462
806.7487604618073
808.7666628360748
812.4822890758514
814.7111139297485
817.1322932243347
818.7977933883667
820.9055547714233
822.7399821281433
826.4845902919769
828.7179744243622
831.2931067943573
833.3871104717255
835.7070920467377
837.4946441650391
839.4851260185242
841.1129851341

...

152.97560277581215
154.28053119778633
155.28786924481392
157.06451657414436
159.7436366379261
160.8090491592884
162.20888105034828
165.6788560450077
168.06433048844337
170.00395241379738
172.51142850518227
174.11463513970375
175.56658473610878
176.52579978108406
178.61231163144112
179.67564025521278
181.4157419502735
183.3495912849903
185.18086710572243
186.30822983384132
188.64759793877602
190.5313505232334
191.7809752523899
193.33658745884895
194.2381121814251
196.22706684470177
198.35586628317833
199.86103639006615
200.44730696082115
202.13145825266838
203.58456394076347
204.94476339221
206.27986428141594
209.41934844851494
211.88786885142326
213.11303409934044
214.46970781683922
215.71039721369743
217.09852609038353
221.34973153471947
223.28147742152214
224.97464391589165
226.16994699835777
229.05736669898033
229.95050767064095
232.02623036503792
234.36020520329475
236.03554359078407
236.62349697947502
238.06182047724724
239.38340291380882
240.4635302722454
241.6758310496807
243.78

233.649251639843
234.77180808782578
236.3391426205635
237.17454141378403
237.95483201742172
238.33564966917038
241.5312289595604
243.28125590085983
245.1498811841011
247.75616532564163
249.4320769906044
251.24056082963943
252.65856355428696
254.01502627134323
255.9003981947899
258.7177318930626
259.95288068056107
262.5344894528389
263.87782150506973
265.6901186108589
267.13291746377945
268.08679991960526
268.986121237278
270.2676423192024
270.8926982283592
271.77500343322754
274.0541031360626
274.80506098270416
276.32090866565704
277.42673230171204
279.47526264190674
282.87680983543396
283.7896466255188
285.18661868572235
286.7260105609894
288.2167944908142
290.36563086509705
291.68971490859985
292.5327779054642
293.7769786119461
296.65794241428375
298.28367948532104
300.2885866165161
301.0886006951332
302.26390451192856
303.07334500551224
304.2309542298317
305.52257496118546
306.73903769254684
307.9677594304085
309.44714123010635
309.9935572743416
310.4589569568634
311.89866280555725


313.6096348166466
315.47543197870255
316.2982369661331
317.2089239358902
318.0602924823761
319.1740976572037
321.3309370279312
322.44970417022705
323.6879633665085
325.6457725763321
327.3541247844696
330.51330280303955
331.74303901195526
334.1611326932907
335.3877899646759
337.3409034013748
338.90172696113586
340.09357476234436
340.74416023492813
342.5451974272728
343.7905281186104
345.9193187355995
346.8135580420494
347.46081179380417
348.7774813771248
351.8414035439491
353.4079810976982
355.1020364165306
356.55510383844376
359.2673731446266
360.5902619957924
361.9758229851723
364.15705066919327
365.34286254644394
365.76311153173447
366.8022182583809
368.7265024781227
369.15081626176834
369.626337826252
370.9942573904991
373.0168623328209
373.8337759375572
375.56111735105515
376.5394004583359
378.0928683280945
378.9789688587189
380.35764038562775
384.16989743709564
384.76907527446747
385.63347762823105
387.240005671978
389.35341519117355
391.1956755518913
392.26386135816574
392.692214

411.31108662486076
413.91619822382927
415.1696067750454
416.4641636312008
418.6057365834713
419.7075546681881
421.8840146958828
423.10648462176323
423.9856995046139
424.83599469065666
426.1099292933941
426.83189955353737
427.78561213612556
429.4133955538273
430.24904730916023
430.9125235378742
431.5254638493061
433.9303107559681
434.9898444712162
435.8110716044903
436.71186926960945
439.14405539631844
439.76878133416176
440.3320894539356
441.0602205693722
442.0818760097027
444.12647983431816
445.9328133761883
447.3077517449856
448.9288153350353
449.22707962989807
449.7798812389374
450.3629903793335
451.92848765850067
453.19951927661896
456.26907908916473
456.9700580239296
457.512631714344
459.07284992933273
459.9549939632416
461.2282962799072
461.8871220946312
463.10794323682785
464.5843885540962
467.7788673043251
469.0343515276909
469.75510454177856
470.5671426653862
471.4149369597435
472.10229086875916
473.6012783050537
474.3907206058502
475.7577049732208
477.18456768989563
478.26433

465.09305173158646
466.3914403319359
468.063798725605
469.1600312590599
470.2754959464073
471.5726816058159
473.367447912693
477.0285287499428
477.8039398789406
478.6872959136963
Cost at epoch 19: 478.6872959136963 <<<<<<<<<
1.848847508430481
2.8266422748565674
3.9167537689208984
5.10646915435791
6.147822380065918
8.363313913345337
9.974758267402649
11.274402141571045
12.68845546245575
13.372605919837952
13.824057698249817
14.720420897006989
16.677424371242523
17.10533034801483
18.13926100730896
19.79921281337738
20.328867077827454
22.669642567634583
22.8983037173748
23.959903687238693
24.689911514520645
25.576329857110977
27.439275175333023
30.112324625253677
31.49284389615059
32.09636601805687
33.028976291418076
33.906996697187424
34.409697622060776
36.20685288310051
37.35177502036095
39.8877817094326
42.11124262213707
42.59531754255295
44.24321931600571
45.32041209936142
47.154077231884
47.976691126823425
50.674872279167175
53.38722813129425
54.532278656959534
55.98187077045441
56.4

In [22]:
acosts2 = model(X_train,y_train,lr=0.0001,num_epochs=20,print_cost=True)

INFO:tensorflow:Restoring parameters from ../../data/dinorunner/models_stochastic/yolo_model.ckpt
Cost at epoch 1: 416.6941807568073
Cost at epoch 2: 386.68143866956234
Cost at epoch 3: 378.32866479456425
Cost at epoch 4: 343.08506932109594
Cost at epoch 5: 339.0946616306901
Cost at epoch 6: 317.533089466393
Cost at epoch 7: 296.5765845179558
Cost at epoch 8: 274.88218408450484
Cost at epoch 9: 258.43429947271943
Cost at epoch 10: 236.25231055915356
Cost at epoch 11: 218.09567795321345
Cost at epoch 12: 233.04172767698765
Cost at epoch 13: 201.85080390237272
Cost at epoch 14: 180.0573816727847
Cost at epoch 15: 187.44365946576
Cost at epoch 16: 175.07797471992671
Cost at epoch 17: 167.88070767000318
Cost at epoch 18: 157.1568614114076
Cost at epoch 19: 154.72874666098505
Cost at epoch 20: 147.87491068430245


In [23]:
acosts3 = model(X_train,y_train,lr=0.00005,num_epochs=20,print_cost=True)

INFO:tensorflow:Restoring parameters from ../../data/dinorunner/models_stochastic/yolo_model.ckpt
Cost at epoch 1: 97.44555820524693
Cost at epoch 2: 75.57211612444371
Cost at epoch 3: 68.51453684363514
Cost at epoch 4: 62.45480118878186
Cost at epoch 5: 61.12613560631871
Cost at epoch 6: 63.348744846880436
Cost at epoch 7: 62.58778141532093
Cost at epoch 8: 57.57068779040128
Cost at epoch 9: 52.80365648679435
Cost at epoch 10: 51.83584228530526
Cost at epoch 11: 50.74157548323274
Cost at epoch 12: 46.72836657380685
Cost at epoch 13: 46.874521852936596
Cost at epoch 14: 40.61734129069373
Cost at epoch 15: 36.78459636121988
Cost at epoch 16: 35.74804900912568
Cost at epoch 17: 36.77534045698121
Cost at epoch 18: 32.15307931229472
Cost at epoch 19: 34.26022667763755
Cost at epoch 20: 31.081302797421813


In [ ]:
acosts4 = model(X_train,y_train,lr=0.00001,num_epochs=20,print_cost=True)

In [ ]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32,shape=(None,448,448,3))
Z = forward_pass(X)
saver = tf.train.Saver()
with tf.Session() as sess:
    saver = tf.train.import_meta_graph("../../data/dinorunner/models_stochastic/yolo_model.ckpt.meta")
    saver.restore(sess, "../../data/dinorunner/models_stochastic/yolo_model.ckpt")
    aX = X_train[0]
    aX.shape = (1,448,448,3)
    aZ = sess.run(Z,feed_dict={X:aX})